In [1]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==0.24.1

Found existing installation: scikit-learn 1.0.2
Uninstalling scikit-learn-1.0.2:
  Successfully uninstalled scikit-learn-1.0.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.9 MB 4.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.24.1 which is incompatible.


In [2]:
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 3.4 MB/s 


##Loading and Preprocessing

In [3]:
# mount drive
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [1]:
import nltk
import numpy as np
from scipy.sparse import save_npz, load_npz
import pandas as pd

from tqdm import tqdm

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score,precision_score,recall_score
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import LabelPowerset
from sklearn.naive_bayes import GaussianNB
import pickle

In [3]:
data_dir = "/content/gdrive/MyDrive/Data/"

x_train = pd.read_csv(data_dir + "x_train_final.csv")["question"].to_numpy()
x_test = pd.read_csv(data_dir + "x_val_final.csv")["question"].to_numpy()

y_train = load_npz(data_dir + "y_train_final.npz").toarray()
y_test = load_npz(data_dir + "y_val_final.npz").toarray()

In [7]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
y_train.data

In [4]:
final_train_size = x_train.shape[0] // 5
final_test_size = x_test.shape[0] // 5

x_train_final = x_train[:final_train_size]
y_train_final = y_train[:final_train_size]

x_test_final = x_test[-final_test_size:]
y_test_final = y_test[-final_test_size:]

print(x_train_final.shape, y_train_final.shape, x_test_final.shape, y_test_final.shape)

(77733,) (77733, 500) (19433,) (19433, 500)


In [10]:
# tfidf_vect = TfidfVectorizer(min_df=0.00009,max_features=200000,smooth_idf=True,norm='l2',\
#                tokenizer=lambda x : x.split(),sublinear_tf=False, ngram_range=(1,3) )

In [11]:
# x_train_vectors = tfidf_vect.fit_transform(x_train_final)
# x_test_vectors = tfidf_vect.transform(x_test_final)

In [12]:
# save_npz(data_dir+'x_train_vectors.npz', x_train_vectors)
# save_npz(data_dir+'/x_test_vectors.npz', x_test_vectors)

In [5]:
x_train_vectors = load_npz(data_dir + "x_train_vectors.npz")
x_test_vectors = load_npz(data_dir + "x_test_vectors.npz")

In [6]:
print("Dimensions of train data X:",x_train_vectors.shape, "Y :",y_train_final.shape)
print("Dimensions of test data X:",x_test_vectors.shape,"Y:",y_test_final.shape)

Dimensions of train data X: (77733, 94625) Y : (77733, 500)
Dimensions of test data X: (19433, 94625) Y: (19433, 500)


In [7]:
def get_predictions(x_test, model, chunksize):
    
    predictions = []
    
    n_samples = x_test.shape[0]
    n_iter = n_samples // chunksize
    last_iter = 0
    
    if (n_samples % chunksize):
        n_iter += 1
        last_iter = n_samples % chunksize
    
    with tqdm(total=n_samples, desc="Progress") as pbar:
        
        for i in range(n_iter-1):
            sub_samples = x_test[int(i*chunksize): int((i+1)*chunksize)]
            preds = model.predict(sub_samples)
            predictions.extend(preds.tolist())
            pbar.update(preds.shape[0])
        
        sub_samples = x_test[-last_iter:]
        preds = model.predict(sub_samples)
        predictions.extend(preds.tolist())
        pbar.update(preds.shape[0])
    
    predictions = np.array(predictions)
    return predictions

##ML KNN

In [ ]:
from skmultilearn.adapt import MLkNN

classifier = MLkNN(k=3)

# train
classifier.fit(x_train_vectors,y_train_final)

MLkNN(k=3)

In [10]:
predictions = classifier.predict(x_test_vectors)

In [15]:
print("accuracy ", metrics.accuracy_score(y_test_final,predictions))
print("f1 score ", metrics.f1_score(y_test_final, predictions, average="macro"))

accuracy  0.07502701590078732
f1 score  0.18543139306436576


##RAkELd


In [18]:
from sklearn.naive_bayes import GaussianNB
from skmultilearn.ensemble import RakelD

classifier = RakelD(
    base_classifier=GaussianNB(),
    base_classifier_require_dense=[True, True],
    labelset_size=4
)

# train
classifier.fit(x_train_vectors[:2000],y_train_final[:2000])

LabelSpacePartitioningClassifier(classifier=LabelPowerset(classifier=GaussianNB(),
                                                          require_dense=[True,
                                                                         True]),
                                 clusterer=RandomLabelSpaceClusterer(allow_overlap=False,
                                                                     cluster_count=125,
                                                                     cluster_size=4),
                                 require_dense=[False, False])

In [9]:
predictions = classifier.predict(x_test_vectors)

In [19]:
print("accuracy ", metrics.accuracy_score(y_test_final[:100],predictions))
print("macro f1 score ",metrics.f1_score(y_test_final[:100],predictions, average='macro'))

accuracy  0.40945682351259727
macro f1 score  0.3525682495223461


##CC

###One vs Rest

In [15]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
classifier = OneVsRestClassifier(SGDClassifier(loss='log', alpha=0.00001, penalty='l1'), n_jobs=-1)
classifier.fit(x_train_vectors,y_train_final)

OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05, loss='log',
                                            penalty='l1'),
                    n_jobs=-1)

In [12]:
#predictions = classifier.predict(x_test_vectors)

predictions = get_predictions(x_test_vectors, classifier, chunksize=1000)

Progress: 100%|█████████████████████████| 19433/19433 [00:05<00:00, 3353.68it/s]


In [13]:
print("accuracy ", metrics.accuracy_score(y_test_final,predictions))
print("macro f1 score ",metrics.f1_score(y_test_final,predictions, average='macro'))

accuracy  0.21592137086399424
macro f1 score  0.3114089766564016
